<a href="https://colab.research.google.com/github/RushiBShinde/ThinkForge-IE-643-project/blob/main/%20IE643_Final_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Unsloth (for Llama 3.1 8B 4-bit)
!pip install "unsloth[colab-new]" -q

# 2. Install other key libraries
# sentence-transformers: for Specter-2 ranking model
# arxiv: to fetch papers from arxiv.org
# pypdf: to read and extract text from PDF files
# gradio: to build the user interface
# pandas: to create the downloadable CSV file
# PyMuPDF: (fitz) for PDF text searching and highlighting
# nltk: for robust sentence splitting
!pip install sentence-transformers arxiv pypdf gradio pandas PyMuPDF nltk -q

# 3. Create directories for our files
!mkdir -p temp_pdfs
!mkdir -p highlighted_results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.5/511.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 19.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
# Download the NLTK model for sentence tokenization
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Cell 2: Load Highlighting LLM (Llama 3.1 8B 4-bit)

In [ ]:
import torch
from unsloth import FastLanguageModel

# Define the model to use for extraction
extraction_model_name = "unsloth/llama-3.1-8b-Instruct"

print("--- Loading Extraction LLM (Llama 3.1 8B 4-bit) ---")

# Load the model in 4-bit using Unsloth
extraction_model, extraction_tokenizer = FastLanguageModel.from_pretrained(
    model_name = extraction_model_name,
    max_seq_length = 32768,  # Use a high value for chunking
    dtype = None,
    load_in_4bit = True,
)

# === CRITICAL Llama 3.1 FIX ===
extraction_tokenizer.add_special_tokens({
    "eos_token": "<|eot_id|>",
})
extraction_model.config.pad_token_id = extraction_tokenizer.eos_token_id
# ================================

# Move the model to the GPU for inference
FastLanguageModel.for_inference(extraction_model)

print("--- Extraction LLM Loaded Successfully ---")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
--- Loading Extraction LLM (Llama 3.1 8B 4-bit) ---
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

--- Extraction LLM Loaded Successfully ---


Cell 3: Load Ranking Model (Specter-2)

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# Define the model for retrieval/ranking
ranking_model_name = "Multi-qa-mpnet-base-dot-v1"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"--- Loading Ranking Model (Specter-2) on {device} ---")

# Load the model
ranking_model = SentenceTransformer(ranking_model_name, device=device)

print("--- Ranking Model Loaded Successfully ---")

#-------------------------------------------------------------------------------
# # --- MODIFIED CODE FOR 4-BIT RANKING ---
# from sentence_transformers import SentenceTransformer
# import torch
# from transformers import BitsAndBytesConfig # <--- 1. IMPORT THIS

# # Define the model for retrieval/ranking
# ranking_model_name = "Multi-qa-mpnet-base-dot-v1"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# print(f"--- Loading Ranking Model (Specter-2) on {device} IN 4-BIT ---")

# # --- 2. Define the 4-bit config ---
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for computation
# )
# # ------------------------------------

# # --- 3. Load the model with the config ---
# # We pass the bnb_config into the 'model_kwargs' dictionary.
# # The SentenceTransformer library will pass this down to the
# # underlying transformers model.
# ranking_model = SentenceTransformer(
#     ranking_model_name,
#     device=device,
#     model_kwargs={'quantization_config': bnb_config}
# )
# # ---------------------------------------

# print("--- Ranking Model Loaded Successfully (in 4-bit) ---")
# print("WARNING: Ranking accuracy will be SEVERELY degraded.")

--- Loading Ranking Model (Specter-2) on cuda ---


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- Ranking Model Loaded Successfully ---


Cell 4: Helper Functions - Data Ingestion (Updated)

In [ ]:
import arxiv
import pypdf
import os
import re
from typing import List, Dict, Any

# --- ArXiv Processing ---

def parse_arxiv_ids(input_data: str) -> List[str]:
    """Parses a list of URLs or IDs from a string (file content or textbox)."""
    ids = []
    lines = input_data.splitlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Regex to find arxiv ID
        match = re.search(r'(\d{4}\.\d{4,5}(v\d+)?)', line)
        if match:
            ids.append(match.group(1))
    return list(set(ids)) # Return unique IDs

def process_arxiv_links(arxiv_links: str, file_obj) -> (List[Dict[str, Any]], str):
    """
    Fetches papers from ArXiv from either a textbox or a file.
    Saves PDFs to ./temp_pdfs/ for highlighting.
    """
    doc_store = []
    status = ""

    # 1. Get list of IDs
    arxiv_ids = []
    if arxiv_links:
        arxiv_ids.extend(parse_arxiv_ids(arxiv_links))

    if file_obj:
        try:
            with open(file_obj.name, 'r') as f:
                file_content = f.read()
            arxiv_ids.extend(parse_arxiv_ids(file_content))
        except Exception as e:
            status += f"Error reading ArXiv file: {e}\n"

    if not arxiv_ids:
        status += "No valid ArXiv IDs found.\n"
        return doc_store, status

    arxiv_ids = list(set(arxiv_ids))
    print(f"Found ArXiv IDs: {arxiv_ids}")

    # 2. Fetch from ArXiv
    try:
        search = arxiv.Search(id_list=arxiv_ids, max_results=len(arxiv_ids))
        client = arxiv.Client()
        results = list(client.results(search))
        status += f"Fetched {len(results)} papers from ArXiv.\n"

        for paper in results:
            pdf_filename = f"{paper.get_short_id()}.pdf"
            pdf_path = os.path.join("./temp_pdfs/", pdf_filename)

            try:
                # Download PDF to our temp folder
                paper.download_pdf(dirpath="./temp_pdfs/", filename=pdf_filename)

                # Extract text
                with open(pdf_path, 'rb') as f:
                    reader = pypdf.PdfReader(f)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() or ""

                doc_store.append({
                    "id": paper.get_short_id(),
                    "title": paper.title,
                    "authors": [str(a) for a in paper.authors],
                    "url": paper.entry_id,
                    "text": text,
                    "pdf_path": pdf_path  # Save the path for highlighting
                })
                status += f"Processed: {paper.title}\n"

            except Exception as e:
                status += f"Error processing paper {paper.entry_id}: {e}\n"
                if os.path.exists(pdf_path):
                    os.remove(pdf_path) # Clean up failed download

    except Exception as e:
        status += f"Error during ArXiv search: {e}\n"

    return doc_store, status

# --- Manual PDF Upload Processing ---

def process_uploaded_pdfs(file_objs: List) -> (List[Dict[str, Any]], str):
    """
    Processes manually uploaded PDF files and extracts text.
    Saves a copy in ./temp_pdfs/ for highlighting.
    """
    if file_objs is None:
        return [], ""

    status = f"Processing {len(file_objs)} manually uploaded PDFs.\n"
    doc_store = []

    for file_obj in file_objs:
        try:
            filename = os.path.basename(file_obj.name)
            pdf_path = os.path.join("./temp_pdfs/", filename)

            # We must read from the temp path (file_obj.name)
            # and write a copy to our persistent ./temp_pdfs/ folder
            with open(file_obj.name, 'rb') as f_in:
                # Save a copy for later
                with open(pdf_path, 'wb') as f_out:
                    f_out.write(f_in.read())

                # Rewind and read for text extraction
                f_in.seek(0)
                reader = pypdf.PdfReader(f_in)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""

            title = reader.metadata.title if reader.metadata and reader.metadata.title else filename
            authors = [reader.metadata.author] if reader.metadata and reader.metadata.author else ["Unknown"]

            doc_store.append({
                "id": filename.replace('.pdf', ''),
                "title": title,
                "authors": authors,
                "url": "N/A (Manual Upload)",
                "text": text,
                "pdf_path": pdf_path # Save the path for highlighting
            })
            status += f"Processed: {filename}\n"
        except Exception as e:
            status += f"Error processing manual PDF {file_obj.name}: {e}\n"

    return doc_store, status

Cell 5: Helper Functions - Ranking & Chunking (Updated)

In [ ]:
from sentence_transformers.util import cos_sim
import numpy as np

# --- Chunking for Ranking  ---
def chunk_text_for_ranking(text: str, chunk_size: int = 512, stride: int = 128) -> List[str]:
    """Splits text into overlapping chunks based on word count."""
    words = text.split()
    chunks = []
    for i in range(0, len(words), stride):
        chunk = words[i : i + chunk_size]
        if len(chunk) < 10: # Skip small chunks
            continue
        chunks.append(" ".join(chunk))
        if i + chunk_size >= len(words):
            break
    return chunks

# --- Scoring (Updated) ---
def get_doc_score(
    query_embedding,
    doc_text: str,
    ranking_model,
    k_chunks: int
) -> (float, List[tuple]):
    """
    Calculates a relevance score for a document based on the new
    reciprocal weights logic (top-k chunks).
    """

    # 1. Chunk the document (512/128)
    chunks = chunk_text_for_ranking(doc_text, chunk_size=512, stride=128)
    if not chunks:
        return 0.0, []

    # 2. Embed all chunks
    chunk_embeddings = ranking_model.encode(
        chunks,
        convert_to_tensor=True,
        show_progress_bar=False
    )

    # 3. Calculate cosine similarity
    similarities = cos_sim(query_embedding, chunk_embeddings)[0].cpu().numpy()

    # 4. Get all chunk scores, sorted high-to-low
    chunk_scores = sorted(zip(similarities, chunks), key=lambda x: x[0], reverse=True)

    # 5. Aggregate score using NEW Reciprocal Weights logic
    doc_score = 0.0

    # Get the top k scores from the *sorted* list
    top_k_scores = [score for score, text in chunk_scores[:k_chunks]]

    for i, score in enumerate(top_k_scores):
        doc_score += score / (i + 1)

    return float(doc_score), chunk_scores

Cell 6: Helper Functions - Extraction (LLM) (NEW/Updated)

In [ ]:
import json
import gradio as gr
from typing import List, Dict, Any

# --- Chunking for Extraction (Llama 3.1) ---
def chunk_text_for_llm(
    text: str,
    tokenizer,
    max_tokens: int = 7500,
    overlap_tokens: int = 200
) -> List[str]:
    """
    Splits text into overlapping chunks based on *token count*
    to safely fit into the LLM's context.
    """
    tokens = tokenizer.encode(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_text = tokenizer.decode(tokens[start:end], skip_special_tokens=True)
        chunks.append(chunk_text)
        start += (max_tokens - overlap_tokens)
    return chunks

# --- System Prompt (Unchanged) ---
EXTRACTION_SYSTEM_PROMPT = """You are an expert analysis assistant. Your task is to analyze a query and a passage.
First, find **all** exact sentences or phrases from the passage that **directly answer the query**.
Second, for each extracted text, provide a relevance_score as a **float between 0.00 and 10.00** (e.g., 8.75) based on how well it answers the query.
Extracted text snippets can be short (20 tokens) or long (400 tokens) as needed. Focus on accuracy.
If no part of the passage answers the query, the 'results' list should be empty (`[]`).
Provide your answer *only* in the following JSON format:
{
  "results": [
    {
      "relevance_score": <float_score_0.00_to_10.00>,
      "extracted_text": "<first_extracted_text>"
    },
    {
      "relevance_score": <float_score_0.00_to_10.00>,
      "extracted_text": "<second_formatted_text>"
    }
  ]
}"""

# --- Function for "Fast Mode" (Only processes one chunk) ---
def get_snippets_from_single_chunk(query: str, chunk: str) -> List[Dict[str, Any]]:
    """
    Runs the LLM on *only one* chunk of text. Used for "Fast Mode".
    """
    messages = [
        {"role": "system", "content": EXTRACTION_SYSTEM_PROMPT},
        {"role": "user", "content": f"Query: \"{query}\"\n\nPassage: \"{chunk}\""}
    ]
    prompt = extraction_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = extraction_tokenizer([prompt], return_tensors="pt").to(device)

    outputs = extraction_model.generate(
        **inputs, max_new_tokens=1024,
        pad_token_id=extraction_tokenizer.eos_token_id,
        eos_token_id=extraction_tokenizer.eos_token_id,
        do_sample=False
    )
    text_output = extraction_tokenizer.batch_decode(
        outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True
    )[0]

    # Parse JSON
    all_snippets = []
    try:
        json_start = text_output.find('{')
        json_end = text_output.rfind('}') + 1
        if json_start != -1 and json_end != -1:
            json_str = text_output[json_start:json_end]
            parsed_json = json.loads(json_str)
            if 'results' in parsed_json and isinstance(parsed_json['results'], list):
                all_snippets.extend(parsed_json['results'])
    except json.JSONDecodeError:
        print(f"Warning: Could not parse JSON from LLM output (fast-mode chunk).")

    return all_snippets

# --- Main (Full Mode) Extraction Function (RESTORED TO ORIGINAL) ---
def get_all_snippets_from_paper(
    query: str,
    full_passage: str,
    progress: gr.Progress
) -> List[Dict[str, Any]]:
    """
    This is the original, full-power function. It runs the LLM
    on the *entire* paper, chunking it as needed.
    """
    # 1. Chunk the full passage into LLM-safe chunks
    llm_chunks = chunk_text_for_llm(
        full_passage,
        extraction_tokenizer,
        max_tokens=7500,
        overlap_tokens=200
    )
    all_snippets = []
    total_chunks = len(llm_chunks)

    print(f"Running FULL extraction on {total_chunks} chunk(s) for this paper...")

    # 2. Loop and run the LLM on *each* chunk
    for i, chunk in enumerate(llm_chunks):
        progress(i / total_chunks, desc=f"Analyzing chunk {i+1}/{total_chunks}")

        messages = [
            {"role": "system", "content": EXTRACTION_SYSTEM_PROMPT},
            {"role": "user", "content": f"Query: \"{query}\"\n\nPassage: \"{chunk}\""}
        ]
        prompt = extraction_tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        inputs = extraction_tokenizer([prompt], return_tensors="pt").to(device)

        outputs = extraction_model.generate(
            **inputs, max_new_tokens=1024,
            pad_token_id=extraction_tokenizer.eos_token_id,
            eos_token_id=extraction_tokenizer.eos_token_id,
            do_sample=False
        )
        text_output = extraction_tokenizer.batch_decode(
            outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True
        )[0]

        # 3. Parse JSON from this chunk's output
        try:
            json_start = text_output.find('{')
            json_end = text_output.rfind('}') + 1
            if json_start != -1 and json_end != -1:
                json_str = text_output[json_start:json_end]
                parsed_json = json.loads(json_str)
                if 'results' in parsed_json and isinstance(parsed_json['results'], list):
                    all_snippets.extend(parsed_json['results'])
            else:
                 print(f"Warning: No JSON object found in LLM output for chunk {i+1}.")

        except json.JSONDecodeError:
            print(f"Warning: Could not parse JSON from LLM output for chunk {i+1}.")

    # 4. Return all aggregated snippets
    return all_snippets

Cell 7: Helper Function - PDF Highlighting (NEW)

In [ ]:
import fitz # PyMuPDF
import os
import re
import numpy as np
from sentence_transformers.util import cos_sim
from typing import List

# Use regex to split sentences (replaces NLTK)
sent_splitter = re.compile(r'(?<=[.!?])\s+')

def highlight_pdf(
    original_pdf_path: str,
    snippets: List[str],
    output_pdf_path: str,
    model, # <-- We pass in the ranking_model
    threshold: float # <-- We pass in the similarity threshold
) -> (str, str):
    """
    Finds text snippets in a PDF using SEMANTIC SENTENCE-vs-SENTENCE search.
    """
    if not os.path.exists(original_pdf_path):
        return "Error: Original PDF not found.", None

    try:
        # 1. Take all LLM snippets, break them into "clean" sentences
        all_llm_sents = []
        if not snippets:
            return "Warning: No snippets were provided to highlight.", None

        for snippet in snippets:
            all_llm_sents.extend(sent_splitter.split(snippet))

        # Filter out tiny fragments
        all_llm_sents = [s.strip() for s in all_llm_sents if len(s.strip()) > 10]
        if not all_llm_sents:
             return "Warning: No valid sentences found in LLM snippets.", None

        # 2. Embed the "clean" LLM sentences
        llm_sent_embeddings = model.encode(all_llm_sents, show_progress_bar=False)

        doc = fitz.open(original_pdf_path)
        highlights_added = 0

        # 3. Go through the PDF page by page
        for page_num, page in enumerate(doc):
            page_text = page.get_text()
            if not page_text:
                continue

            # 4. Extract all "messy" sentences from the page
            pdf_sents = sent_splitter.split(page_text)
            pdf_sents_clean = [
                s.replace('\n', ' ').strip() for s in pdf_sents
                if len(s.strip()) > 10
            ]
            if not pdf_sents_clean:
                continue

            # 5. Embed all "messy" PDF sentences from this page
            pdf_sent_embeddings = model.encode(pdf_sents_clean, show_progress_bar=False)

            # 6. Calculate similarity matrix
            # Shape: (num_llm_sents, num_pdf_sents_on_page)
            sim_matrix = cos_sim(llm_sent_embeddings, pdf_sent_embeddings)

            # 7. Find the best score for *each* PDF sentence

            # --- THIS IS THE FIX ---
            # .max(axis=0) returns a (values, indices) tuple
            # We must select the 'values' (the first element) before calling .cpu()
            values, indices = sim_matrix.max(axis=0)
            best_scores_for_pdf_sents = values.cpu().numpy()
            # ---------------------

            # 8. Get all PDF sentences that passed the threshold
            matching_indices = np.where(best_scores_for_pdf_sents > threshold)[0]

            if not matching_indices.any():
                continue

            # 9. Highlight all found sentences on the page
            for idx in matching_indices:
                sentence_to_find = pdf_sents_clean[idx]

                rects = page.search_for(sentence_to_find)
                for rect in rects:
                    page.add_highlight_annot(rect)
                    highlights_added += 1

        # --- End of loops ---

        if highlights_added == 0:
            status = "Warning: Found 0 semantic matches. Try lowering the similarity threshold."
        else:
            status = f"Successfully added {highlights_added} semantic highlight(s)."

        doc.save(output_pdf_path)
        doc.close()

        return status, output_pdf_path

    except Exception as e:
        return f"Error during PDF highlighting: {e}", None

Cell 8: Gradio UI & Pipeline Functions (NEW)

In [ ]:
import gradio as gr
import pandas as pd
import json

# --- Helper function to create the main JSON export ---
def create_json_export(extraction_state, query):
    """Formats the entire extraction state into a single JSON serializable dict."""
    export_data = {
        "query": query,
        "papers": []
    }
    sorted_ranks = sorted(extraction_state.keys())
    for rank in sorted_ranks:
        paper_data = extraction_state[rank]
        doc = paper_data['doc']
        paper_export = {
            "rank": doc['rank'],
            "title": doc['title'],
            "authors": doc['authors'],
            "url": doc['url'],
            "extracted_snippets": paper_data['all_snippets']
        }
        export_data["papers"].append(paper_export)
    return export_data

# --- Pipeline for Tab 1  ---
def run_ingestion_pipeline(arxiv_links, arxiv_file, pdf_files, progress=gr.Progress()):
    progress(0, desc="Starting paper ingestion...")
    doc_store = []
    status = ""
    if arxiv_links or arxiv_file:
        progress(0.2, desc="Processing ArXiv papers...")
        arxiv_docs, arxiv_status = process_arxiv_links(arxiv_links, arxiv_file)
        doc_store.extend(arxiv_docs)
        status += arxiv_status
    if pdf_files:
        progress(0.6, desc="Processing uploaded PDFs...")
        pdf_docs, pdf_status = process_uploaded_pdfs(pdf_files)
        doc_store.extend(pdf_docs)
        status += pdf_status
    if not doc_store:
        status += "\nError: No documents were successfully processed."
    status += f"\n\nTotal documents processed: {len(doc_store)}"
    progress(1, desc="Ingestion complete.")
    return doc_store, status

# --- Pipeline for Tab 2  ---
def run_ranking_pipeline(doc_store, query, k_chunks, progress=gr.Progress()):
    """
    This saves the 'top_chunks' to the doc object,
    which Fast Mode will use.
    """
    if not doc_store:
        return pd.DataFrame(), [], "Error: No papers processed. Please go to Tab 1.", {}
    if not query:
        return pd.DataFrame(), [], "Error: Please enter a query.", {}

    progress(0, desc="Embedding query with Specter-2...")
    query_embedding = ranking_model.encode(query, convert_to_tensor=True)
    ranked_results = []
    total_docs = len(doc_store)
    progress(0.2, desc=f"Ranking {total_docs} documents...")

    for i, doc in enumerate(doc_store):
        progress(0.2 + (0.8 * (i / total_docs)), desc=f"Ranking doc {i+1}/{total_docs}...")
        doc_score, top_chunks = get_doc_score(
            query_embedding, doc['text'], ranking_model, k_chunks
        )
        doc['score'] = doc_score
        doc['top_chunks'] = top_chunks  # This is for Fast Mode
        ranked_results.append(doc)

    ranked_results.sort(key=lambda x: x['score'], reverse=True)

    df_data = []
    for i, doc in enumerate(ranked_results):
        doc['rank'] = i + 1
        df_data.append({
            "Rank": doc['rank'], "Score": doc['score'], "Title": doc['title'],
            "Authors": ", ".join(doc['authors']), "URL": doc['url']
        })
    df = pd.DataFrame(df_data)
    app_state = {doc['rank']: doc for doc in ranked_results}
    progress(1, desc="Ranking complete.")
    return df, ranked_results, "Ranking complete.", app_state

# --- Pipeline for Tab 3 (Extraction)  ---
def run_extraction_pipeline(
    ranked_results,
    query,
    top_k_papers,
    fast_mode,
    progress=gr.Progress()
):
    """
    Pipeline for Tab 3 Button.
    Now correctly uses 'fast_mode' to decide which function to call.
    """
    if not ranked_results:
        return "Error: No papers ranked.", {}, None, gr.Dropdown(choices=[]), gr.File(value=None, visible=False)

    papers_to_process = ranked_results[:top_k_papers]
    extraction_state = {}
    dropdown_choices = []

    total_papers = len(papers_to_process)
    print(f"--- Starting Extraction Pipeline (Fast Mode: {fast_mode}) ---")

    for i, doc in enumerate(papers_to_process):
        rank = doc['rank']
        title = doc['title']
        progress(i / total_papers, desc=f"Extracting Rank {rank}: {title[:30]}...")

        all_snippets = []
        if fast_mode:
            print(f"Running Fast Mode for Rank {rank}")
            if not doc.get('top_chunks') or not doc['top_chunks']:
                first_chunk = chunk_text_for_llm(doc['text'], extraction_tokenizer, 7500, 200)[0]
                all_snippets = get_snippets_from_single_chunk(query, first_chunk)
            else:
                top_chunk_text = doc['top_chunks'][0][1]
                all_snippets = get_snippets_from_single_chunk(query, top_chunk_text)
        else:
            print(f"Running Full Mode for Rank {rank}")
            all_snippets = get_all_snippets_from_paper(query, doc['text'], progress)

        all_snippets.sort(key=lambda x: x['relevance_score'], reverse=True)
        extraction_state[rank] = {
            "doc": doc, "all_snippets": all_snippets
        }
        dropdown_choices.append(f"Rank {rank}: {title}")

    progress(1, desc="Extraction complete!")
    print("--- Extraction Pipeline Complete ---")

    json_export_data = create_json_export(extraction_state, query)
    json_export_path = "./highlighted_results/extraction_results.json"
    with open(json_export_path, 'w') as f:
        json.dump(json_export_data, f, indent=2)

    status = (
        f"Extraction complete for {total_papers} papers! "
        "Select a paper from the dropdown below to see details."
    )

    return (
        status,
        extraction_state,
        json_export_path,
        gr.Dropdown(choices=dropdown_choices, value=dropdown_choices[0], label="Select a Paper to View", interactive=True),
        gr.File(value=json_export_path, visible=True)
    )

# --- Function for Tab 3 Dropdown (UPDATED) ---
def show_paper_snippets(
    selected_paper_str: str,
    extraction_state: dict,
    top_k_snippets: int
):
    """
    Triggered when a user selects a paper from the dropdown.
    This function ONLY displays the text snippets.
    """
    if not selected_paper_str or not extraction_state:
        return "No paper selected or no data."
    try:
        selected_rank = int(selected_paper_str.split(":")[0].replace("Rank ", ""))
    except:
        return "Error parsing selected paper."
    if selected_rank not in extraction_state:
        return f"Error: Could not find Rank {selected_rank} in extracted data."

    paper_data = extraction_state[selected_rank]
    doc = paper_data['doc']
    all_snippets = paper_data['all_snippets']
    snippets_to_show = all_snippets[:top_k_snippets]

    snippet_md = f"### Relevant Snippets for: {doc['title']}\n\n"
    if not snippets_to_show:
        snippet_md += "*No relevant snippets found by the LLM for this paper.*"

    for snippet in snippets_to_show:
        snippet_md += (
            f"> {snippet['extracted_text']}\n"
            f"> *(Relevance: {snippet['relevance_score']:.2f}/10.00)*\n\n"
        )
    return snippet_md

# --- NEW Function for the "Highlight PDF" Button ---
def run_pdf_highlighting(
    selected_paper_str: str,
    extraction_state: dict,
    top_k_snippets: int,
    highlight_threshold: float
):
    """
    Triggered when the user clicks the "Generate Highlighted PDF" button.
    This function ONLY does the highlighting and returns a file/status.
    """
    if not selected_paper_str or not extraction_state:
        return "Error: No paper selected.", None
    try:
        selected_rank = int(selected_paper_str.split(":")[0].replace("Rank ", ""))
    except:
        return "Error parsing selected paper.", None
    if selected_rank not in extraction_state:
        return f"Error: Could not find Rank {selected_rank} in extracted data.", None

    paper_data = extraction_state[selected_rank]
    doc = paper_data['doc']
    all_snippets = paper_data['all_snippets']

    # Get the *same* snippets the user is currently seeing
    snippets_to_show = all_snippets[:top_k_snippets]
    snippet_texts_for_highlighting = [
        s['extracted_text'] for s in snippets_to_show
    ]

    output_pdf_filename = f"HIGHLIGHTED_Rank_{doc['rank']}_{doc['id']}.pdf"
    output_pdf_path = os.path.join("./highlighted_results/", output_pdf_filename)

    # Call the new semantic highlighter from Cell 7
    # It can access 'ranking_model' from the global scope
    status, final_path = highlight_pdf(
        doc['pdf_path'],
        snippet_texts_for_highlighting,
        output_pdf_path,
        ranking_model, # Accesses the globally loaded model
        highlight_threshold
    )

    return status, final_path

# --- Gradio UI (UPDATED) ---
print("Launching Gradio Interface...")

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🤖 Advanced RAG Pipeline for Research Papers
        A step-by-step tool to Ingest, Rank, and Extract relevant information.
        """
    )

    # --- STATE VARIABLES ---
    doc_store = gr.State([])
    ranked_results = gr.State([])
    app_state = gr.State({})
    extraction_state = gr.State({})

    with gr.Tabs():
        # --- TAB 1: INGESTION ---
        with gr.Tab("1. Ingest Papers"):
            gr.Markdown("Add papers using one or more methods, then click 'Process Papers'.")
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Option A: Paste ArXiv Links")
                    arxiv_links_input = gr.Textbox(label="ArXiv Links (one per line)", lines=5)
                with gr.Column():
                    gr.Markdown("### Option B: Upload ArXiv Link File")
                    arxiv_file_input = gr.File(file_types=['.txt', '.csv'])
                    gr.Markdown("### Option C: Upload PDFs Manually")
                    pdf_files_input = gr.File(file_types=['.pdf'], file_count="multiple")
            ingest_btn = gr.Button("Process Papers", variant="primary")
            ingest_status_output = gr.Textbox(label="Ingestion Status", interactive=False)

        # --- TAB 2: RANKING ---
        with gr.Tab("2. Rank Papers"):
            gr.Markdown("Enter your query and ranking parameters, then click 'Rank Papers'.")
            query_input = gr.Textbox(label="Query", placeholder="What is Scaled Dot-Product Attention?")
            k_chunks_slider = gr.Slider(label="Reciprocal Weights K (Top 'k' chunks)", minimum=1, maximum=10, value=5, step=1)
            rank_btn = gr.Button("Rank Papers", variant="primary")
            rank_status_output = gr.Textbox(label="Ranking Status", interactive=False)

            gr.Markdown("### Ranked Papers")
            rank_df_output = gr.DataFrame(
                headers=["Rank", "Score", "Title", "Authors", "URL"],
                datatype=["number", "number","str", "str", "str"],
            )

        # --- TAB 3: EXTRACT & HIGHLIGHT (UPDATED LAYOUT) ---
        with gr.Tab("3. Extract & Highlight"):
            gr.Markdown("Run the extraction on your top-ranked papers.")
            with gr.Row():
                top_k_papers_slider = gr.Slider(
                    label="Top-k Papers to Process with LLM", minimum=1, maximum=20, value=5, step=1
                )
                fast_mode_checkbox = gr.Checkbox(
                    label="🚀 Enable Fast Mode (Tests only the single most relevant chunk)",
                    value=True
                )
            extract_btn = gr.Button("1. Run Extraction on Top-k Papers", variant="primary")
            extract_status_output = gr.Textbox(label="Extraction Status", interactive=False)

            gr.Markdown("---")

            # --- Download All JSON ---
            gr.Markdown("### 1. Download All Results")
            gr.Markdown("This file contains *all* extracted snippets for *all* processed papers.")
            json_download_output = gr.File(label="Download all_results.json", visible=False)

            gr.Markdown("---")

            # --- Per-Paper Results ---
            gr.Markdown("### 2. View Individual Paper Results")
            gr.Markdown("Select a paper, view snippets, then adjust threshold and generate PDF.")

            processed_papers_dropdown = gr.Dropdown(
                label="Select a Processed Paper",
                interactive=False
            )

            top_k_snippets_slider = gr.Slider(
                label="Top-k Snippets to Show", minimum=1, maximum=10, value=5, step=1
            )

            # This box will *only* show the snippets
            snippet_output = gr.Markdown()

            gr.Markdown("### 3. Generate Highlighted PDF")
            highlight_threshold_slider = gr.Slider(
                label="Highlight Similarity Threshold",
                info="Higher (e.g., 0.95) is stricter. Lower (e.g., 0.85) finds more.",
                minimum=0.7, maximum=1.0, value=0.9, step=0.01
            )

            # New dedicated button
            highlight_pdf_btn = gr.Button("2. Generate Highlighted PDF", variant="primary")
            highlight_status_output = gr.Textbox(label="Highlighting Status", interactive=False)
            highlight_pdf_output = gr.File(label="Download Highlighted PDF")

    # --- Wire up the components (NEW LOGIC) ---

    # TAB 1
    ingest_btn.click(
        fn=run_ingestion_pipeline,
        inputs=[arxiv_links_input, arxiv_file_input, pdf_files_input],
        outputs=[doc_store, ingest_status_output]
    )

    # TAB 2
    rank_btn.click(
        fn=run_ranking_pipeline,
        inputs=[doc_store, query_input, k_chunks_slider],
        outputs=[rank_df_output, ranked_results, rank_status_output, app_state]
    )

    # TAB 3
    # "Extract" button
    extract_btn.click(
        fn=run_extraction_pipeline,
        inputs=[
            ranked_results,
            query_input,
            top_k_papers_slider,
            fast_mode_checkbox
        ],
        outputs=[
            extract_status_output,
            extraction_state,
            json_download_output,
            processed_papers_dropdown,
            json_download_output
        ]
    )

    # Dropdown change now *only* shows snippets
    processed_papers_dropdown.change(
        fn=show_paper_snippets,
        inputs=[processed_papers_dropdown, extraction_state, top_k_snippets_slider],
        outputs=[snippet_output]
    )

    # Slider change for snippets *also* updates snippets
    top_k_snippets_slider.release(
        fn=show_paper_snippets,
        inputs=[processed_papers_dropdown, extraction_state, top_k_snippets_slider],
        outputs=[snippet_output]
    )

    # New "Generate PDF" button
    highlight_pdf_btn.click(
        fn=run_pdf_highlighting,
        inputs=[
            processed_papers_dropdown,
            extraction_state,
            top_k_snippets_slider,
            highlight_threshold_slider
        ],
        outputs=[highlight_status_output, highlight_pdf_output]
    )

# Launch the app
demo.launch(debug=True, share=True)

Launching Gradio Interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://93e0f0483ad475e8ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Found ArXiv IDs: ['2510.26167', '2510.26518', '2510.26411', '2510.26270', '2510.26242', '2510.26396', '2510.26384', '2510.26732', '2510.26486', '1706.03762', '2510.26380', '2510.26418', '2307.09288', '1810.04805', '2510.26402', '2510.26481', '2510.26702', '2510.26238', '2510.26784', '2510.26550', '2510.26606', '2510.26493', '2510.26721', '2510.26658', '2510.26603', '2510.26309', '2510.26346', '2510.26374', '2510.26752']
Found ArXiv IDs: ['2510.26143']
Found ArXiv IDs: ['2501.05433', '2503.01179', '2510.26732', '2505.00252', '1706.03762', '2503.00008', '2307.09288', '2505.00313', '2511.02809', '1810.04805', '2510.26702', '2510.26784', '2511.02787', '2502.00049', '2510.26721', '2510.26658', '2505.00067', '2503.00536', '2510.26752']
--- Starting Extraction Pipeline (Fast Mode: False) ---
Running Full Mode for Rank 1
Running FULL extraction on 2 chunk(s) for this paper...
Running Full Mode for Rank 2
Running FULL extraction on 4 chunk(s) for this paper...
Running Full Mode for Rank 3
Runni